# Resumo

Neste notebook, uma rede neural é criada, treinada e avaliada para a detecção de câncer (classificar um tumor entre maligno ou benigno) de mama a partir de imagens de células.<br>
Após isso, uma nova rede, com 1 camada extra e mais parâmetros personalizados, é treinada e avaliada na mesma base da dados.<br>
A conclusão é que a complexidade extra levou a um pior desempenho, provavelmente por conta de overfitting.<br>
Além disso, é feita a visualização dos pesos da segunda rede.<br><br>
O dataset utilizado é o Breast Cancer Winsconsin, do repositório UC Irvine.<br>
Possui 569 amostras, cada uma com 32 variáveis (1 ID, 1 target e 30 atributos).<br>
Target: 0 para begnino, 1 para maligno (classificação binária).<br>
Não há dados faltantes.<br>

# Importação de recursos

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential

2024-08-20 14:01:42.497180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Importação dos dados

In [3]:
X = pd.read_csv("Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/entradas_breast.csv") #variáveis independentes
y = pd.read_csv("Deep Learning com Python de A a Z/Parte 1 - Redes Neurais Artificiais/classificação binária/saidas_breast.csv") #variável dependente

# Preparação dos conjuntos de treino e teste

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Construindo a rede neural sequencial

In [5]:
network = Sequential([ #cria uma instância de rede sequencial
    tf.keras.layers.InputLayer(shape=(30,)), #cria uma camada de entrada com 30 neuronios
    tf.keras.layers.Dense(units = 16, activation = 'relu', kernel_initializer = 'random_uniform'), #cria uma camada oculta densa (todos os neuronios ligados a todos da camada de entrada) com 16 neuronios ((inputLayer + outputLayer)/2), com ativação relu e um inicializador dos pesos com distribuição uniforme
    tf.keras.layers.Dense(units = 1, activation='sigmoid'), #cria uma camada de saída densa com 1 neurônio e ativação sigmoide (ideal para classificação binária)
])

In [49]:
network.summary() #imprime informações sobre a rede neural, que indica um total de 513 parâmetros (pesos)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,541 (6.02 KB)

 Trainable params: 513 (2.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,028 (4.02 KB)

# Configurando a Rede Neural

In [7]:
network.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['BinaryAccuracy']) #otimizador adam, função de custo do tipo Crossentropy Binária e métrica de precisão binária

# Treinando a rede neural

In [8]:
network.fit(X_train, y_train, batch_size=10, epochs = 100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - BinaryAccuracy: 0.4533 - loss: 27.4303
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - BinaryAccuracy: 0.6313 - loss: 0.9664
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - BinaryAccuracy: 0.7206 - loss: 0.6749
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.7492 - loss: 0.5290
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.7958 - loss: 0.4192
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.8046 - loss: 0.4002
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.7787 - loss: 0.4455
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.8542 - loss: 0.3532
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - BinaryAccuracy: 0.8738 - loss: 0.3136
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.8536 - loss: 0.3461
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - BinaryAccuracy: 0.8650 - loss: 0.2

# Testando a rede neural, método 1

In [9]:
predictions = network.predict(X_test) #faz e retorna as previsões (em forma de array de listas) para as entradas em X_test
predictions = predictions > 0.5 #converte cada elemento de predictions com base na booleana que indica se satisfaz ou não essa condição

accuracy = accuracy_score(y_test, predictions) #calcula a precisão das previsões em predictions com base nas respostas corretas em y_test
matrix = confusion_matrix(y_test, predictions) #faz a matriz de confusão das previsões

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [10]:
#imprime os resultados:
print(accuracy)
print(matrix)

0.9090909090909091
[[44  4]
 [ 9 86]]


# Testando a rede neural, método 2

In [50]:
evaluation = network.evaluate(X_test, y_test) #retorna uma lista com as taxas de erro e acerto
evaluation

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - BinaryAccuracy: 0.9024 - loss: 0.2121 


[0.21092000603675842, 0.9090909361839294]

# Mais camadas e parâmetros

In [55]:
network2 = Sequential([
    tf.keras.layers.InputLayer(shape = (30,)),
    tf.keras.layers.Dense(units = 16, activation = 'relu', kernel_initializer = 'random_uniform'),
    tf.keras.layers.Dense(units = 16, activation = 'relu', kernel_initializer = 'random_uniform'),
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
])

network2.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785 (3.07 KB)

 Trainable params: 785 (3.07 KB)

 Non-trainable params: 0 (0.00 B)

## Criando uma instância personalizada do Adam

In [56]:
adam1 = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)

A linha de código acima cria uma instância do otimizador Adam com taxa de aprendizado (learning_rate) padrão de 0.001 e adiciona um clipvalue (valor limite para os pesos) de 0.5.

## Configurando e treinando a nova rede

In [57]:
network2.compile(optimizer=adam1, loss='binary_crossentropy', metrics=['binary_accuracy'])
network2.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_accuracy: 0.6724 - loss: 1.9425
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.6614 - loss: 0.4986
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7209 - loss: 0.4590
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8146 - loss: 0.4053
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7478 - loss: 0.4917
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7963 - loss: 0.4704
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7061 - loss: 0.6812
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8192 - loss: 0.4460
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8545 - loss: 0.3961
Epoch 10/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8724 - loss: 0.4092
Epoch 11/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8474 -

## Testando a rede, método 1

In [58]:
predictions2 = network2.predict(X_test)
predictions2 = predictions2 > 0.5
accuracy2 = accuracy_score(y_test, predictions2)
matrix2 = confusion_matrix(y_test, predictions2)
print(accuracy2)
print(matrix2)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
0.8811188811188811
[[33 15]
 [ 2 93]]


## Testando a rede, método 2

In [59]:
evaluation2 = network2.evaluate(X_test, y_test)
evaluation2

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8632 - loss: 0.8656  


[0.7150412797927856, 0.881118893623352]

Houve uma piora grande na taxa de erros e uma piora pequena na taxa de acertos.<br>
Como se trata de uma base de dados simples, um número menor de camadas se mostrou adequado.<br>
A camada extra adicionada em network2 possivelmente gerou overfitting.

# Visulizando os pesos

In [60]:
weights0 = network2.layers[0].get_weights() #retorna os pesos da 1a camada oculta
weights0

[array([[-1.66836336e-01, -2.06480339e-01,  2.24111617e-01,
         -2.99726091e-02,  1.03764869e-02, -4.68090503e-03,
          3.86847067e-03,  1.47085786e-01, -8.19281954e-03,
         -8.05261508e-02,  3.47777717e-02, -5.25517669e-03,
         -7.09078787e-03, -7.47382864e-02,  4.58794087e-02,
         -2.06128769e-02],
        [-1.55770734e-01, -2.66613096e-01,  1.18703647e-02,
          1.16191417e-01, -4.00768258e-02, -8.40151012e-02,
          2.52400279e-01,  1.35309681e-01,  1.99366942e-01,
          2.08703622e-01, -1.75391827e-02,  1.30634159e-01,
          1.76165241e-03, -1.35993585e-01, -1.09086387e-01,
          8.55695363e-03],
        [-1.17616110e-01,  3.54578085e-02, -8.38527307e-02,
          1.97903216e-02, -5.78318052e-02, -1.84057131e-02,
          1.73832402e-01, -3.45883369e-02,  2.69445237e-02,
          9.61170867e-02, -6.91312551e-02,  3.71405110e-02,
          1.64611265e-02,  6.27809688e-02, -1.65201023e-01,
          3.62495035e-02],
        [ 3.3040925

In [26]:
len(weights0)

2

É um array composto por 2 outros arrays: um contendo os pesos entre os neurônios da camada de entrada e os da escondida e outro contendo os pesos entre a unidade de bias e os neurônios da camada escondida.

In [43]:
len(weights0[1])

16

Ou seja, temos 1 unidade de bias ligada a cada um dos 16 neurônios da camada escondida.

In [44]:
len(weights0[0])

30

São 30 arrays, cada 1 contendo os pesos das 16 ligações entre 1 neurônio da camada de entrada com cada neurônio da camada oculta.

In [45]:
len(weights0[0][0])

16

In [46]:
weights1 = network2.layers[1].get_weights() #retorna os pesos da segunda camada oculta

In [62]:
len(weights1[0])

16

São 16 arrays, cada um contendo os pesos das 16 ligações de um neurônio da primeira camada oculta com cada neurônio da segunda.

In [63]:
len(weights1[1])

16

São os pesos das 16 ligações entre a unidade de bias e cada neurônio da segunda camada escondida.

In [64]:
weights2 = network2.layers[2].get_weights() #retorna o array de pesos referente à camada de saída

In [65]:
len(weights2)

2

In [67]:
len(weights2[0])

16

Dessa vez, cada 1 dos 16 arrays vai ter apenas 1 elemento, já que só há um neurônio na camada de saída para se ligar.<br>

In [69]:
len(weights2[1])

1